# CNNs in Depth

## Convolutional layers in depth

Definitions:
- dense layers = fully connected layers, meaning that the nodes are connected to every node in the previous layer
- locally-connected layers (convolutional layers) = meaning that the nodes are only connected to a small subset of the previous layer nodes

Kernels for color images:
- grayscale images uses a matrix of `kernel_size` x `kernel_size` numbers
- color images uses a 3d filter of `kernel_size` x `kernal_size` x 3 for RGB colors

"Channels" indicate the feature maps of convolutional layers: a convolutional layer that takes feature maps with a depth of 64 and outputs 128 feature maps is said to have 64 channels as input and 128 as outputs.

### The number of parameters in a convolutional layer
The number of parameters in a convolutional layer is the number of filters times the number of input feature maps times the size of the kernel squared plus the number of kernels:
- $n_k$​ : number of filters in the convolutional layer
- $k$ : height and width of the convolutional kernel
- $c$ : number of feature maps produced by the previous layer (or number of channels in input image)

There are k times k times c weights per filter plus one bias per filter, so $c*k2 + 1$ parameters. The convolutional layer is composed of $n_k$​ filters, so the total number of parameters in the convolutional layer is: $n_p = n_k (c k^2 + 1)$


## Convolutional Layers in PyTorch

Example code to create a convolutional layer:
```python
from torch import nn
conv1 = nn.Conv2d(in_channels, out_channels, kernel_size)
```

We can also add an activation and dropout function. In case of CNNs, we need to use the 2d version of dropout, which randomly drops some input channel entirely:
```python
conv_block = nn.Sequential(
  nn.Conv2d(in_channels, out_channels, kernel_size),
  nn.ReLU(),
  nn.Dropout2d(p=0.2)
)
```

## Stride and padding

Padding and stride are hyperparameters, or configuration settings, for the filter:
- Padding: Expanding the size of an image by adding pixels at its border
- Stride: Amount by which a filter slides over an image.

There are multiple padding strategies:
- zero-padding strategy is by far the most common: padding values will all be 0
- reflect: padding pixels filled with copies of values in input image taken in opposite order, in a mirroring fashion
- replicate: padding pixels filled with value of closest pixel in input image
- circular: like reflect mode, but image is first flipped horizontally and vertically

## Pooling layers

Also for pooling layers you can set the padding and stride paramters.

Different padding strategies:
- max-pooling: take the maximum value in the window = mostly used for images
- average-pooling: take mean average of all the values in the window

Example code for max and average pooling:
```python
from torch import nn
nn.MaxPool2d(kernel_size, stride)
nn.AvgPool2d(window_size, stride)
```

Definitions:
- Kerel size: size of the side of the convolutional kernel
- Window size: size of the window considered during pooling
- Stride: Step size of the convolutional kernel or of the pooling window when moving over the input image
- Padding: Border to add to an input image before the convolution operation is performed